In [109]:
# import package 
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import math

In [110]:
my_link = pd.read_excel('../input_network/Nguyen–Dupuis_network.xlsx', sheet_name='link_info')
my_node = pd.read_excel('../input_network/Nguyen–Dupuis_network.xlsx', sheet_name='node_info')

In [111]:
my_link['flow']=100
my_link

,link_id,from,to,type,attribute,c_a,l_a,t_0,travel,fare_rate,f_bus,walk,wait,congestion,penalty,flow
0,1,N1,N2,physical,auto,300,3.0000,0,0,0,0,0,0,0,0,100
1,2,N4,N1,physical,auto,200,3.0000,1,1,0,0,0,0,0,0,100
2,3,N2,N5,physical,auto,200,3.0000,2,2,0,0,0,0,0,0,100
3,4,N2,N7,physical,auto,300,7.0000,3,3,0,0,0,0,0,0,100
4,5,N3,N4,physical,auto,500,3.0000,4,4,0,0,0,0,0,0,100
5,6,N4,N5,physical,auto,500,3.0000,5,5,0,0,0,0,0,0,100
6,7,N5,N6,physical,auto,500,3.0000,6,6,0,0,0,0,0,0,100
7,8,N6,N7,physical,auto,500,3.0000,7,7,0,0,0,0,0,0,100
8,9,N3,N8,physical,auto,300,6.0000,8,8,0,0,0,0,0,0,100
9,10,N4,N8,physical,auto,300,3.0000,9,9,0,0,0,0,0,0,100


In [112]:
def compute_t0(x, v_auto=60, v_hailing=40, v_transit=30): # minutes
    """comupute free flow time for different modes

    Args:
        x ([pd.DataFrame]): [link info]
        v_auto (int, optional): [auto speed]. Defaults to 60.
        v_hailing (int, optional): [ride-hailing speed]. Defaults to 40.
        v_transit (int, optional): [transit speed]. Defaults to 30.

    Returns:
        [pd.Series]: [t_0]
    """

    if x['attribute'] == 'auto':
        return (x['l_a'] / v_auto) * 60
    
    elif x['attribute'] == 'ride-hailing':
        return (x['l_a'] / v_hailing) *60
    
    elif x['attribute'] == 'transit':
        return (x['l_a'] / v_transit)*60
    
    elif x['attribute'] == 'P&R':
        return 5

    else:
        return 0


In [113]:
def compute_travel_time(x):
    """compute the travel time

    Args:
        x ([pd.DataFrame]): [link info]

    Returns:
        [pd.Series]: [travel time]
    """
    
    if x['attribute'] in ['auto', 'P&R']:
        return x['t_0'] * (1 + 0.15 * (x['flow'] / x['c_a']) ** 4) # BPR function

    elif x['attribute'] == 'transit':
        return x['t_0'] * x['f_bus']/x['f_bus'] # compute travel time based on bus frequency 

    else:
        return x['t_0'] # the travel time for other modes is same as the free flow time

In [114]:
def compute_walk_time(x, v_walk=5.6):
    """compute_walk_time

    Args:
        x ([pd.DataFrame]): [link info]
        v_walk (float, optional): [walk speed]. Defaults to 5.6.

    Returns:
        [pd.Series]: [walk speed]
    """
    
    if x['attribute'] in ['transit-transfer', 'board_transit']: #by bus or bus transfer 
        return (x['l_a'] / v_walk) * 60

    elif x['attribute'] in ['board_ride-hailing', 'R&T']: # by ride-hailing
        return 3
    
    elif x['attribute'] in ['P&R']: # auto transfer bus
        return 5
    
    else: # other modes
        return 0

In [115]:
def compute_wait_time(x, n_idle):
    """compute_wait_time

    Args:
        x ([pd.DataFrame]): [link info]
        n_idle ([int]): [the number of idle ride-hailing vehicles]

    Returns:
        [pd.Series]: [wait time]
    """

    if x['attribute'] in ['transit-transfer','P&R','R&T','board_transit']:
        return 60 / x['f_bus']
    
    elif x['attribute'] in ['board_ride-hailing']:
        A = 2 # waiting time coefficient 
        return (A / (1 + n_idle**0.5)) * 60
    
    else:
        return 0


In [116]:
def compute_congesition_time(x):
    """compute_congesition_time for transit link

    Args:
        x ([pd.DataFrame]): [link info]

    Returns:
        [pd.Series]: [congestion time]
    """

    if x['attribute'] in ['transit']:
        return  60 * ((0.5 * x['flow'] / (120 * x['f_bus'])))**3
    
    else:
        return 0

In [117]:
def compute_penalty(x, penalty=12):
    """add a transfer penalty for transfer link

    Args:
        x ([pd.DataFrame]): [link info]
        penalty (int, optional): [transfer penalty]. Defaults to 12.

    Returns:
        [pd.Series]: [transfer penalty]
    """

    if x['attribute'] in ['transit-transfer','P&R','R&T']:
        return penalty
    
    else:
        return 0

In [118]:
def compute_lamda_0_auto(fuel_consumption=6.46, driving_speed=50, fuel_price=9.2):
    """compute_lamda_0_auto: compute auto fare rate for auto link

    Args:
        fuel_consumption (float, optional): [fuel consumuption per 100 km]. Defaults to 6.46.
        driving_speed (int, optional): [average auto speed]. Defaults to 50.
        fuel_price (float, optional): [fuel price]. Defaults to 9.2.

    Returns:
        [int]: [fare rate: RMB/h]
    """

    lamda_0_auto = round((fuel_consumption / 100 * fuel_price * driving_speed), 2) # unit: RMB/h, fare rate for auto
    return lamda_0_auto

In [119]:
def compute_fare(x,lamda_0_auto, lamda_0_hailing, lamda_0_PR):
    """compute fare for each link

    Args:
        x ([pd.DataFrame]): [link info]
        lamda_0_hailing ([int]): [fare rate for ride-hailing service]
        lamda_0_PR ([int]): [park charge for P&R]

    Returns:
        [pd.Series]: [computed_fare]
    """

    if x['attribute'] in ['auto']:
        return lamda_0_auto * (x['travel'] / 60)
    
    elif x['attribute'] == 'ride-hailing':
        return lamda_0_hailing * x['travel']
    
    elif x['attribute'] == 'P&R':
        return lamda_0_PR
    
    else:
        return 0

In [120]:
def compute_link_time(link_table, n_idle, lamda_0_auto, lamda_0_hailing, lamda_0_PR):
    """compute link time

    Args:
        link_table ([pd.DataFrame]): [link info]
        n_idle ([int]): [idle ride-hailing vehiles]
        lamda_0_auto ([int]): [fare rate for auto link]
        lamda_0_hailing ([int]): [fare rate for ride-hailing link]
        lamda_0_PR ([int]): [fare rate for P&R link]

    Returns:
        [type]: [description]
    """
    
    link_table['t_0'] = link_table.apply(compute_t0, axis = 1)
    link_table['travel'] = link_table.apply(compute_travel_time, axis = 1)
    link_table['fare_rate'] = link_table.apply(compute_fare, 
                                               lamda_0_hailing=lamda_0_hailing, 
                                               lamda_0_PR=lamda_0_PR,
                                               lamda_0_auto=lamda_0_auto,
                                               axis = 1)
    link_table['walk'] = link_table.apply(compute_walk_time, axis = 1)
    link_table['wait'] = link_table.apply(compute_wait_time, n_idle = n_idle, axis = 1)
    link_table['congestion'] = link_table.apply(compute_congesition_time, axis = 1)
    link_table['penalty'] = link_table.apply(compute_penalty, axis = 1)

    return link_table

In [121]:
n_idle = 100
lamda_0_auto = compute_lamda_0_auto()
lamda_0_hailing = 0
lamda_0_PR = 12
compute_link_time(my_link, n_idle, lamda_0_auto, lamda_0_hailing, lamda_0_PR)

,link_id,from,to,type,attribute,c_a,l_a,t_0,travel,fare_rate,f_bus,walk,wait,congestion,penalty,flow
0,1,N1,N2,physical,auto,300,3.0000,3.0000,3.0056,1.4888,0,0.0000,0.0000,0.0000,0,100
1,2,N4,N1,physical,auto,200,3.0000,3.0000,3.0281,1.4999,0,0.0000,0.0000,0.0000,0,100
2,3,N2,N5,physical,auto,200,3.0000,3.0000,3.0281,1.4999,0,0.0000,0.0000,0.0000,0,100
3,4,N2,N7,physical,auto,300,7.0000,7.0000,7.0130,3.4738,0,0.0000,0.0000,0.0000,0,100
4,5,N3,N4,physical,auto,500,3.0000,3.0000,3.0007,1.4864,0,0.0000,0.0000,0.0000,0,100
5,6,N4,N5,physical,auto,500,3.0000,3.0000,3.0007,1.4864,0,0.0000,0.0000,0.0000,0,100
6,7,N5,N6,physical,auto,500,3.0000,3.0000,3.0007,1.4864,0,0.0000,0.0000,0.0000,0,100
7,8,N6,N7,physical,auto,500,3.0000,3.0000,3.0007,1.4864,0,0.0000,0.0000,0.0000,0,100
8,9,N3,N8,physical,auto,300,6.0000,6.0000,6.0111,2.9775,0,0.0000,0.0000,0.0000,0,100
9,10,N4,N8,physical,auto,300,3.0000,3.0000,3.0056,1.4888,0,0.0000,0.0000,0.0000,0,100


In [126]:
import link_cost

In [127]:
my_n_idle = 50
lamda_0_auto = link_cost.compute_lamda_0_auto()
lamda_0_hailing = 3
lamda_0_PR= 10

In [128]:
link_cost.compute_link_time(my_link, my_n_idle, lamda_0_auto, lamda_0_hailing, lamda_0_PR)

,link_id,from,to,type,attribute,c_a,l_a,t_0,travel,fare_rate,f_bus,walk,wait,congestion,penalty,flow
0,1,N1,N2,physical,auto,300,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
1,2,N4,N1,physical,auto,200,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
2,3,N2,N5,physical,auto,200,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
3,4,N2,N7,physical,auto,300,7.0000,7.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
4,5,N3,N4,physical,auto,500,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
5,6,N4,N5,physical,auto,500,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
6,7,N5,N6,physical,auto,500,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
7,8,N6,N7,physical,auto,500,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
8,9,N3,N8,physical,auto,300,6.0000,6.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
9,10,N4,N8,physical,auto,300,3.0000,3.0000,NaN,NaN,0,0.0000,0.0000,0.0000,0,100
